<a href="https://colab.research.google.com/github/dani3lkim/Poke/blob/main/streamlit_app_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import streamlit as st
import requests
import pandas as pd
import matplotlib.pyplot as plt

# Your backend URL
API = "https://8000-m-s-30qtpp4kk02pl-d.us-east1-1.prod.colab.dev"

st.set_page_config(page_title="Restaurant Dashboard", layout="wide")
st.title("Restaurant Daily Performance")

# -----------------------------
# Fetch daily metrics
# -----------------------------
@st.cache_data
def get_metrics():
    try:
        r = requests.get(f"{API}/metrics/daily", timeout=10)
        df = pd.DataFrame(r.json())

        if not df.empty:
            numeric_cols = [
                "total_sales",
                "labor_expenses",
                "food_expenses",
                "other_expenses",
                "profit",
            ]
            for col in numeric_cols:
                df[col] = pd.to_numeric(df[col], errors="coerce").fillna(0.0)

        return df

    except Exception as e:
        st.warning(f"Could not fetch metrics: {e}")
        return pd.DataFrame()

df = get_metrics()

# -----------------------------
# KPI CARDS
# -----------------------------
st.subheader("Today's KPIs")

if not df.empty:
    latest = df.iloc[-1]
    col1, col2, col3, col4, col5 = st.columns(5)

    col1.metric("Sales", f"${latest['total_sales']:.2f}")
    col2.metric("Labor", f"${latest['labor_expenses']:.2f}")
    col3.metric("Food", f"${latest['food_expenses']:.2f}")
    col4.metric("Other", f"${latest['other_expenses']:.2f}")
    col5.metric("Profit", f"${latest['profit']:.2f}")
else:
    st.info("No data yet. Add today's metrics below.")

# -----------------------------
# Trend Graph
# -----------------------------
st.subheader("Daily Trends")

if not df.empty:
    df["date"] = pd.to_datetime(df["date"])

    fig, ax = plt.subplots(figsize=(10, 4))
    ax.plot(df["date"], df["total_sales"], label="Sales")
    ax.plot(df["date"], df["labor_expenses"], label="Labor")
    ax.plot(df["date"], df["food_expenses"], label="Food")
    ax.plot(df["date"], df["other_expenses"], label="Other")
    ax.plot(df["date"], df["profit"], label="Profit", linewidth=3)

    ax.legend()
    ax.set_xlabel("Date")
    ax.set_ylabel("Amount ($)")
    fig.autofmt_xdate()

    st.pyplot(fig)
else:
    st.write("No data to plot yet.")

# -----------------------------
# Add Daily Metrics Form
# -----------------------------
st.subheader("Add Daily Metrics")

with st.form("daily_form"):
    date = st.date_input("Date")
    sales = st.number_input("Total Sales", min_value=0.0, step=10.0)
    labor = st.number_input("Labor Expenses", min_value=0.0, step=10.0)
    food = st.number_input("Food Expenses", min_value=0.0, step=10.0)
    other = st.number_input("Other Expenses", min_value=0.0, step=10.0)

    submitted = st.form_submit_button("Submit")

    if submitted:
        payload = {
            "date": str(date),
            "total_sales": float(sales),
            "labor_expenses": float(labor),
            "food_expenses": float(food),
            "other_expenses": float(other),
        }

        try:
            resp = requests.post(f"{API}/log/daily", json=payload, timeout=10)
            if resp.status_code == 200:
                st.success("Daily metrics added!")
                st.cache_data.clear()
            else:
                st.error(f"API error: {resp.status_code} {resp.text}")
        except Exception as e:
            st.error(f"Could not reach backend: {e}")

# -----------------------------
# History Table
# -----------------------------
st.subheader("History")

if not df.empty:
    st.dataframe(df.sort_values("date", ascending=False))
else:
    st.write("No historical data yet.")

ModuleNotFoundError: No module named 'streamlit'